# Lab 1, Assignment 1 - Simulating an acrobot 

##  EL2820 Modelling of Dynamical Systems

Group members: INSERT GROUP MEMBERS


Group number: INSERT GROUP NUMBER

In this home-work assignment, you will simulate an "acrobot" (see figure below), which is a simple robot consisting of two bars, of masses $m_1$, $m_2$ and lengths $l_1$, $l_2$, respectively, joined by an elbow. The top bar has one of its ends hooked to a fixed support. A motor is connected to the elbow between the two bars, and it can provide an external torque $\tau$. The figure also shows the coordinate system used to describe the positions of the joints and the centers of mass of the bars (denoted by the circles in the middle of each bar), as well as the angle that each bar makes with respect to the negative $y$-axis.

<center> <img src="img.jpg" width="400"> </center>

**The goal of this assignment is to derive a model of an acrobot, and to use it to complete a simulation code.** To this end, we will consider the acrobot as two rigid bodies, whose position and orientation will be described by the coordinates of their center of mass and the angles $\theta_1$ and $\theta_2$. The constraints will be modelled as very stiff springs. The resulting model will not have a minimal number of states, but it can be simplified using the techniques covered in Topic 4 (**not** used in this assignment).

**Note:** Use only techniques covered in the course (no Lagrangian/Hamiltonian mechanics nor bond graphs). The provided answers should be well motivated.

## Instructions

1. Inspect the code below. You will be asked later to modify only one function named `step`.
2. Derive a model of the acrobot, based on the parameters stated in the code below, by solving the tasks below sequentially. This model will subsequently be used to simulate the dynamics of the acrobot.
3. When you consider the assignment done, submit the notebook on Canvas. To facilitate the grading, please *don't* rename the notebook. That is, submit a file named `acrobot.ipynb`.

### Task 1 - Derivation of the dynamical system

### Task 1a

*Write down the equations for balance of linear momentum in each bar (in $x$ and $y$ dimensions), in terms of the components of the force $F_s$ exerted by the support on bar 1, and the force $T$ exerted by bar 2 on bar 1. Do not forget the gravitational force exerted on each bar.*

**Write your answer in the cell below.**  (Hint: you can write equations in "Latex mode", for example, $\dot{x} = \dots$; to show them, execute this block to compile the text.)

### Task 1b
*Write down the equations for balance of angular momentum of bars 1 and 2, in terms of the torques generated by $F_s$ and $T$.*

**Write your answer in the cell below.** 

### Task 1c
*Write down expressions for the $x-$ and $y-$ components of the position of the center of mass of bars 1 and 2, in terms of linear momentum of those bars.*

**Write your answer in the cell below.** 

### Task 1d
*Write down equations for the angles of the bars 1 and 2, with respect to the vertical position, in terms of their angular momenta and their moments of inertia $J_1$ and $J_2$.*

**Write your answer in the cell below.** 

### Task 1e

*Derive expressions for $F_s$ and $T$, by considering the two bars as very stiff springs (of constant $1/\varepsilon$, with $0 < \varepsilon \ll 1$). Hint: compute the $x-$ and $y-$coordinates of the ends of each bar; the components of $F_s$ and $T$ should be proportional to the distance between the components of these ends.*

**Write your answer in the cell below.** 

### Task 2 - Implement the dynamics
*Implement your model in the function `step` (by replacing the lines "# FILL IN!"), given in the Code section below. This function should use the global variables `x1`, `y1`, `x2`, `y2`, `p1x`, `p2x`, `p1y`, `p2y`, `L1`, `L2`, `th1` and `th2` (defined in the code) as state variables and update them via the derivatives you provided in Task 1. To update the state variables, use the Forward Euler discretization scheme for ODE's.*

### Task 3 - Run the simulation
*Run each code block (following the corresponding instructions), and play with the simulation. Save a picture of a pendulum state you find interesting by pressing the s-key. For instance, you can try to get the rods to point upwards. To include the screenshot in the report, rerun the cell below after the picture been saved to include it in the submission.* 

<center> <img src="screenshot.jpeg" width="400"> </center>

<center>  <b> Figure 1. </b> A snapshot of the simulated acrobot.<center>

### Task 4 - Validation of the model

*What are the possible equilibrium points of the acrobot? Are they stable, marginally stable or unstable? Does the simulation agree with your answers? Explain. (Do not perform a linearization of the model: a qualitative argument is enough.)*

**Write your answer in the cell below.** 

*Are the dominant time constants of the simulation "reasonable"? (Hint: Find out (on the web or other resources) the period of oscillation of a bar pendulum of length $l$ for small amplitudes. Is it similar to what you observe in the simulations?)*

**Write your answer in the cell below.** 

*Should the total energy of the system be conserved when no external torque is applied? Why? Does this agree with your simulations? Explain.*

**Write your answer in the cell below.** 

## Code

The next code block imports the necessary libraries for executing this notebook.

In [ ]:
"""Do NOT change anything here"""

from numpy import sin, cos
import numpy as np
import time
import pygame

<br>

**The simulation**

You can send the Torque input to the acrobot through Keyboard. 'w' supplies +1 Nm Torque to the joints and 'a' sends -1 Nm Torque to the joints.


Let us initialize the parameters and variables required for simulating this system:

In [ ]:
"""Do NOT change anything here"""

global l1, l2, m1, m2, g, ee, origin, J1, J2

# Parameters of the acrobot
l1     = 10.0                    # Length of the top bar in m
l2     = 10.0                    # Length of the bottom bar in m
m1     = 10.0                    # Mass of the top bar 1 in Kg
m2     = 10.0                    # Mass of the bottom bar 2 in Kg
g      = 9.8                     # Gravitational acceleration in m/s^2
ee     = 1e-7                    # Singular perturbation constant
origin = (0, 0)                  # (x,y) coordination of the origin
J1     = m1*(l1**2)/12           # Moment of inertia of the top bar 
J2     = m2*(l2**2)/12           # Moment of inertia of the bottom bar

# Initial conditions of the variables 
(x1, y1)     = (0.0, -l1/2)      # Location of the centre of mass of the top bar 
(x2, y2)     = (0.0, -l1-(l2/2)) # Location of the centre of mass of the bottom bar
Fsx          = 0.0        
Fsy          = 0.0        
Tx           = 0.0                    
Ty           = 0.0 
p1x          = 0.0               # Linear momentum in x direction of the top bar
p1y          = 0.0               # Linear momentum in y direction of the top bar
p2x          = 0.0               # Linear momentum in x direction of the bottom bar
p2y          = 0.0               # Linear momentum in y direction of the bottom bar
L1           = 0.0               # Angular momentum w.r.t center of mass of the top bar
L2           = 0.0               # Angular momentum w.r.t center of mass of the bottom bar
th1          = 0.0               # Angle of the top bar
th2          = 0.0               # Angle of the bottom bar
tau1         = 1000.0            # Torque input for the top bar
tau2         = 1000.0            # Torque input for the bottom bar
time_elapsed = 0.0               # A counter for the time elapsed

# Colors
black = (0, 0, 0)
white = (255, 255, 255) 
red   = (255, 0, 0) 
green = (0, 255, 0) 
blue  = (0, 0, 128)

<br>

*position():* This function is used to compute the (x, y) coordinates of the Acrobot arms

In [ ]:
"""Do NOT change anything here"""

def position():
    x = np.cumsum([origin[0], l1 * sin(th1), l2 * sin(th2)])
    y = np.cumsum([origin[1], -l1 * cos(th1), -l2 * cos(th2)])
    return (x, y)

<br>

*step():* This function updates the state variables of the acrobot.

In [ ]:
def step(dt):
    """executes one time step of length dt"""
    global Fsx, Fsy, Tx, Ty, x1, y1, x2, y2, p1x, p2x, p1y, p2y, L1, L2, th1, th2, time_elapsed

    # Computation of the forces Fs and T
    # FILL IN!
    
    # Update linear momentum and position of center of mass of the top bar
    # FILL IN!

    # Update the angular momentum and angle of the top bar
    # FILL IN!

    # Update linear momentum and position of center of mass of the bottom bar
    # FILL IN!

    # Update the angular momentum and angle of the bottom bar
    # FILL IN!

    # Update elapsed time
    time_elapsed += dt

<br>
Below is the main simulation cell that iteratively calls the `step`-function and visualize the updates in a plot.

<br>
<br>

*Keyboard commands:*
- 'w' supplies a +1 Nm Torque to the joints 
- 'a' supplies a -1 Nm Torque to the joints 
- 'q' quits the program
- 's' saves the figure as a TGA file "screenshot.tga"

In [ ]:
"""Do NOT change anything here"""

# Main simulation code

dt = 0.0001

pygame.init()
screen_width  = 500
screen_height = 500
screen = pygame.display.set_mode((screen_width, screen_height))
screen.fill(white)
pygame.display.flip()

xs_ini, ys_ini = position()
pygame.display.set_caption('time = %.4f' % time_elapsed)

start_time = time.time()
i = 1
running = True
while(running):
    step(dt)
    if (i*dt)%0.025 == 0:
        i = 0
        xs, ys = position()

        # Drawing
        screen.fill(white)
        pygame.draw.line(screen, blue, (int(screen_width/2+10*xs[0]),int(screen_height/2-10*ys[0])), \
                                       (int(screen_width/2+10*xs[1]),int(screen_height/2-10*ys[1])),4)
        pygame.draw.line(screen, blue, (int(screen_width/2+10*xs[1]),int(screen_height/2-10*ys[1])), \
                                       (int(screen_width/2+10*xs[2]),int(screen_height/2-10*ys[2])),4)
        pygame.draw.circle(screen, red, (int(screen_width/2+10*xs[0]),int(screen_height/2-10*ys[0])),7)
        pygame.draw.circle(screen, blue, (int(screen_width/2+10*xs[1]),int(screen_height/2-10*ys[1])),7)
        pygame.draw.circle(screen, blue, (int(screen_width/2+10*xs[2]),int(screen_height/2-10*ys[2])),7)
        pygame.display.flip()
        
        pygame.display.set_caption('time = %.4f' % time_elapsed)
        pygame.display.update()
        end_time = time.time()
        time.sleep(max(0.025-(end_time-start_time),0))
        start_time = time.time()
        
        # Keyboard handling

        tau1 = 0.0
        tau2 = 0.0
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    pygame.quit()
                    running = False                        
                if event.key == pygame.K_w:
                    tau1 = 1000.0
                    tau2 = 1000.0
                if event.key == pygame.K_a:
                    tau1 = -1000.0
                    tau2 = -1000.0
                if event.key == pygame.K_s:
                    pygame.image.save(screen,'screenshot.jpeg')


    if np.round(time_elapsed, 2) == 1000.00:
        pygame.quit()
        running = False
    i = i + 1